# Character-level RNN in Keras (char-rnn style)
This notebook implements a simple character-level RNN using Keras and TensorFlow backend, inspired by Andrej Karpathy's char-rnn.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import random

## Load and preprocess text data

In [2]:
# Load your text file (make sure 'input.txt' is in the same directory)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower()

# Create character to index mappings
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Sequence generation
seq_length = 40
step = 3
X = []
y = []

for i in range(0, len(text) - seq_length, step):
    X.append([char_to_idx[ch] for ch in text[i:i + seq_length]])
    y.append(char_to_idx[text[i + seq_length]])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

In [3]:
chars

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

Let's check this out. According to the chars list, the text should begin f (18) i (21) r (30) s (31) t (32)

In [9]:
X

array([[18, 21, 30, ..., 18, 33, 30],
       [31, 32,  1, ..., 32, 20, 17],
       [15, 21, 32, ..., 30,  6,  1],
       ...,
       [28,  7,  7, ..., 30, 32,  1],
       [16, 21, 17, ..., 35, 13, 23],
       [ 6,  1, 30, ..., 21, 26, 19]])

In [13]:
print(y[:10])


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0

## Build the model

In [15]:
model = Sequential([
    Embedding(vocab_size, 64),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model

This takes about 30 minutes to run, so for the demo we are just going to load the model below. For your assignment, you'll need to train it yourself.

In [16]:
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10


2025-04-21 18:06:58.762690: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 118971200 exceeds 10% of free system memory.
2025-04-21 18:06:58.886631: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57998460 exceeds 10% of free system memory.


2905/2905 ━━━━━━━━━━━━━━━━━━━━ 211s 72ms/step - loss: 2.4412
Epoch 2/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 208s 71ms/step - loss: 1.8846
Epoch 3/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 208s 72ms/step - loss: 1.7135
Epoch 4/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 207s 71ms/step - loss: 1.6262
Epoch 5/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 206s 71ms/step - loss: 1.5712
Epoch 6/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 207s 71ms/step - loss: 1.5328
Epoch 7/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 261s 71ms/step - loss: 1.5023
Epoch 8/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 204s 70ms/step - loss: 1.4831
Epoch 9/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 204s 70ms/step - loss: 1.4602
Epoch 10/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 204s 70ms/step - loss: 1.4393


In [18]:
model.save('my_model.keras')

## Load the saved model

In [9]:
from tensorflow.keras.models import load_model

model = load_model('my_model.keras')

2025-04-21 23:34:17.758903: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/vscode/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Text generation utilities

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_text(seed, length=300, temperature=1.0):
    generated = seed
    sequence = [char_to_idx[c] for c in seed]

    for _ in range(length):
        x_pred = np.array(sequence[-seq_length:]).reshape(1, -1)
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]

        generated += next_char
        sequence.append(next_index)

    return generated

## Generate text using a random seed

In [12]:
seed = 'these are all the ducks i have to give'
print("Seed:", repr(seed))
print("\nGenerated text:\n")
print(generate_text(seed, 500, temperature=0.5))

Seed: 'these are all the ducks i have to give'

Generated text:

these are all the ducks i have to give him and scance:
that i cannot seemped that are the grace;
and the more with him too lord of such all
and by heart the strength of your counter
than such and gloucester so murder men.

lucentio:
why, that all the fair so soul of the stands.

henry bolingbroke:
and the prince of deliver to be day and heaved
and the prisonal deserved with this company
in the dear not soul purpomer and heaven,
and i must come more to thee and see the fore.

coriolanus:
sir, it is well, but king he's should should p
